Cài đặt thư viện

In [ ]:
# Cài đặt hugging face
!pip install transformer
# Cài đặt thư viện underthesea để thực hiện word segment
!pip install underthesea
# Cài đặt pytorch
!pip install torch
# Cài đặt sklearn
!pip install scikit-learn

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.5 MB/s eta 0:00:00


Load model PhoBERT

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"  # Fix potential MKL error
import torch
import numpy as np
import re
import underthesea
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from sklearn.svm import SVC
from joblib import dump
import json

# Load data from JSON
def load_data_from_json():
    with open('mood_extended.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    texts = []
    labels = []
    for label, sentences in data.items():
        for sentence in sentences:
            texts.append(sentence)
            labels.append(label)
    return texts, labels

# Preprocessing functions (standardize_data, load_stopwords) remain the same


def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base")
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer



def standardize_data(row):
    row = re.sub(r"[.,?]+$-", "", row)
    row = row.replace(",", " ").replace(".", " ").replace(";", " ").replace("“", " ").replace(":", " ").replace("”", " ").replace('"', " ").replace("'", " ").replace("!", " ").replace("?", " ").replace("-", " ").replace("?", " ")
    row = row.strip().lower()
    return row



def load_stopwords():
    sw = []
    with open("vietnamese-stopwords.txt", encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            sw.append(line.replace("\n",""))
    return sw


def make_bert_features(v_text, tokenizer, phobert, sw):  # Pass tokenizer, phobert, and sw as arguments
    v_tokenized = []
    max_len = 100
    for i_text in v_text:
        print("Đang xử lý line = ", i_text)
        line = underthesea.word_tokenize(i_text)
        filtered_sentence = [w for w in line if not w in sw]
        line = " ".join(filtered_sentence)
        line = underthesea.word_tokenize(line, format="text")
        line = tokenizer.encode(line)
        v_tokenized.append(line)

    padded = np.array([i + [1] * (max_len - len(i)) for i in v_tokenized])
    attention_mask = np.where(padded == 1, 0, 1)

    padded = torch.tensor(padded).to(torch.long)  # Ensure long type for input_ids
    attention_mask = torch.tensor(attention_mask).to(torch.long)  # Ensure long type for attention_mask

    with torch.no_grad():
        last_hidden_states = phobert(input_ids=padded, attention_mask=attention_mask)

    v_features = last_hidden_states[0][:, 0, :].numpy()
    return v_features




# Load stopwords and BERT model
print("Chuẩn bị nạp danh sách các từ vô nghĩa (stopwords)...")
sw = load_stopwords()
print("Đã nạp xong danh sách các từ vô nghĩa")

print("Chuẩn bị nạp model BERT....")
phobert, tokenizer = load_bert()
phobert.eval()  # Put the model in evaluation mode
print("Đã nạp xong model BERT.")

# Load data from JSON
print("Chuẩn bị load dữ liệu....")
text, label_text = load_data_from_json()

# Convert string labels to numerical labels (e.g., "vui" -> 0, "buồn" -> 1)
label_mapping = {label: i for i, label in enumerate(set(label_text))}
label = [label_mapping[l] for l in label_text]

print("Đã load dữ liệu xong")



# Create BERT features
print("Chuẩn bị tạo features từ BERT.....")
features = make_bert_features(text, tokenizer, phobert, sw) # Pass necessary arguments
print("Đã tạo xong features từ BERT")



# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=0)

# Train SVM (using simplified parameter tuning for demonstration)
print("Chuẩn bị train model SVM....")

cl = SVC(kernel='linear', probability=True, gamma='auto')  # Use 'auto' for gamma with linear kernel
cl.fit(X_train, y_train)


sc = cl.score(X_test, y_test)
print('Kết quả train model, độ chính xác = ', sc*100, '%')


# Save the model and label mapping
dump(cl, 'save_model.pkl')
dump(label_mapping, 'label_mapping.pkl')
print("Đã lưu model SVM vào file save_model.pkl và label_mapping.pkl")

Chuẩn bị nạp danh sách các từ vô nghĩa (stopwords)...
Đã nạp xong danh sách các từ vô nghĩa
Chuẩn bị nạp model BERT....


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

Đã nạp xong model BERT.
Chuẩn bị load dữ liệu....
Đã load dữ liệu xong
Chuẩn bị tạo features từ BERT.....
Đang xử lý line =  Cho tôi một bài hát vui.
Đang xử lý line =  Bạn có thể gợi ý bài nào sôi động không?
Đang xử lý line =  Cho tôi bài hát nào phù hợp với tâm trạng hạnh phúc.
Đang xử lý line =  Tôi đang vui, bạn gợi ý gì cho tôi không?
Đang xử lý line =  Mình cảm thấy phấn khởi quá, có gì hay ho không?
Đang xử lý line =  Tâm trạng tôi đang thoải mái, không biết nên làm gì bây giờ.
Đang xử lý line =  Tôi đang vui và muốn tìm một chút gì đó sôi động.
Đang xử lý line =  Bây giờ tôi đang cảm thấy hạnh phúc, bạn nghĩ tôi nên nghe nhạc gì?
Đang xử lý line =  Khi tôi vui thì nghe bài nào hợp nhất?
Đang xử lý line =  Hôm nay trời nắng đẹp, tôi thấy vui, cần nghe bài gì phấn khởi.
Đang xử lý line =  Mình vừa hoàn thành công việc, giờ thấy tự hào, cần nghe nhạc vui vẻ.
Đang xử lý line =  Tôi vừa nhận được tin tốt, giờ thấy hạnh phúc, cần nghe bài gì vui tươi.
Đang xử lý line =  Mình đang cả

In [ ]:
# prompt: Sau khi train mô hình ở trên xong và tải xuống thì làm sao để sử dụng mô hình

import os
import torch
import numpy as np
import re
import underthesea
from transformers import AutoModel, AutoTokenizer
from sklearn.svm import SVC
from joblib import load
import json

# Load stopwords and BERT model
def load_stopwords():
    sw = []
    with open("vietnamese-stopwords.txt", encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            sw.append(line.replace("\n",""))
    return sw

def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base")
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer

def standardize_data(row):
    row = re.sub(r"[.,?]+$-", "", row)
    row = row.replace(",", " ").replace(".", " ").replace(";", " ").replace("“", " ").replace(":", " ").replace("”", " ").replace('"', " ").replace("'", " ").replace("!", " ").replace("?", " ").replace("-", " ").replace("?", " ")
    row = row.strip().lower()
    return row

def make_bert_features(v_text, tokenizer, phobert, sw):
    v_tokenized = []
    max_len = 100
    for i_text in v_text:
        line = underthesea.word_tokenize(i_text)
        filtered_sentence = [w for w in line if not w in sw]
        line = " ".join(filtered_sentence)
        line = underthesea.word_tokenize(line, format="text")
        line = tokenizer.encode(line)
        v_tokenized.append(line)

    padded = np.array([i + [1] * (max_len - len(i)) for i in v_tokenized])
    attention_mask = np.where(padded == 1, 0, 1)

    padded = torch.tensor(padded).to(torch.long)
    attention_mask = torch.tensor(attention_mask).to(torch.long)

    with torch.no_grad():
        last_hidden_states = phobert(input_ids=padded, attention_mask=attention_mask)

    v_features = last_hidden_states[0][:, 0, :].numpy()
    return v_features


# Load the saved model and label mapping
model = load('save_model.pkl')
label_mapping = load('label_mapping.pkl')

# Load stopwords and BERT model
sw = load_stopwords()
phobert, tokenizer = load_bert()
phobert.eval()

# Example usage:
input_text = "Hôm nay tôi rất vui" #Example text
input_text = standardize_data(input_text)
features = make_bert_features([input_text], tokenizer, phobert, sw)
predicted_label_index = model.predict(features)[0]

# Reverse label mapping to get the original label
reverse_label_mapping = {v: k for k, v in label_mapping.items()}
predicted_label = reverse_label_mapping[predicted_label_index]

print(f"Predicted label for '{input_text}': {predicted_label}")

Predicted label for 'hôm nay tôi rất vui': vui


In [ ]:
from joblib import load

def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base")
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer

# Load the saved model and label mapping
model = load('save_model.pkl')
label_mapping = load('label_mapping.pkl')

# Inverted label mapping for getting string labels from predictions
inverted_label_mapping = {v: k for k, v in label_mapping.items()}
# Load PhoBERT and tokenizer (only once)
phobert, tokenizer = load_bert()
phobert.eval()  # Put in evaluation mode
sw = load_stopwords()


def predict_sentiment(text):
    """Predicts the sentiment of a given text."""

    # Preprocess the text
    processed_text = standardize_data(text)

    # Create BERT features
    features = make_bert_features([processed_text], tokenizer, phobert, sw)

    # Make prediction
    prediction = model.predict(features)[0]  # Get the predicted class (integer)

    # Get the string label
    sentiment = inverted_label_mapping[prediction]

    return sentiment



# Example usage:
text = "Tôi đang cảm thấy rất vui"
sentiment = predict_sentiment(text)
print(f"Sentiment: {sentiment}")

text = "Bài hát này thật buồn"
sentiment = predict_sentiment(text)
print(f"Sentiment: {sentiment}")


# Example with multiple sentences
sentences = [
    "Hôm nay trời đẹp quá",
    "Bây giờ tôi đang cảm thấy tuyệt vời, bạn nghĩ tôi nên nghe nhạc gì?",
    "Thất vọng",
    "Tôi đang rất buồn và cô đơn",
]

for sentence in sentences:
    sentiment = predict_sentiment(sentence)
    print(f"Sentence: {sentence}, Sentiment: {sentiment}")

Sentiment: vui
Sentiment: buồn
Sentence: Hôm nay trời đẹp quá, Sentiment: vui
Sentence: Bây giờ tôi đang cảm thấy tuyệt vời, bạn nghĩ tôi nên nghe nhạc gì?, Sentiment: vui
Sentence: Thất vọng, Sentiment: buồn
Sentence: Tôi đang rất buồn và cô đơn, Sentiment: buồn


Mô hình wonrax/phobert-base-vietnamese-sentiment



In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

# Tạo pipeline với mô hình phân loại cảm xúc tiếng Việt
pipe = pipeline("text-classification",model="wonrax/phobert-base-vietnamese-sentiment")

# Dữ liệu mẫu để phân tích
texts=[
    "Hôm nay trời đẹp thế",
    "Hôm nay trời cũng đẹp",
    "Hôm nay trời bình thường",
    "Hôm nay trời âm u thế"
]

# Sử dụng pipeline để phân loại cảm xúc
results = pipe(texts)

# In kết quả
for text, result in zip(texts, results):
    print(f"Text: {text}\nSentiment: {result['label']} (Score: {result['score']:.2f})\n")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


Text: Hôm nay trời đẹp thế
Sentiment: POS (Score: 0.98)

Text: Hôm nay trời cũng đẹp
Sentiment: POS (Score: 0.98)

Text: Hôm nay trời bình thường
Sentiment: NEU (Score: 0.89)

Text: Hôm nay trời âm u thế
Sentiment: NEG (Score: 0.92)



In [ ]:
# Dữ liệu mẫu để phân tích
texts=[
    "Cho tôi một bài hát vui tươi",
    "Cho tôi một bài hát bình thường",
    "Tôi đang cảm thấy nhàm chán, cho tôi một bài hát đi"
]

# Sử dụng pipeline để phân loại cảm xúc
results = pipe(texts)

# In kết quả
for text, result in zip(texts, results):
    print(f"Text: {text}\nSentiment: {result['label']} (Score: {result['score']:.2f})\n")


Text: Cho tôi một bài hát vui tươi
Sentiment: POS (Score: 0.99)

Text: Cho tôi một bài hát bình thường
Sentiment: NEU (Score: 0.90)

Text: Tôi đang cảm thấy nhàm chán, cho tôi một bài hát đi
Sentiment: NEG (Score: 0.97)



# Stop word
